# H_Player

In [1]:
import pandas as pd
from datetime import datetime
import sqlite3
import hashlib
import ssl
from urllib.error import HTTPError
ssl._create_default_https_context = ssl._create_unverified_context

Delete H_Player if it exists.

In [2]:
import sqlite3
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='H_Player'
connection=sqlite3.connect(database=data_base)
cursor = connection.cursor()
drop_query = f"DROP TABLE IF EXISTS {table_name};"
cursor.execute(drop_query)
connection.commit()
connection.close()

Prepare the source data from https://afltables.com/afl/stats/biglists/bg10.txt



In [3]:

# Function to hash combined columns with NaN handling
def hash_row(row):
    combined = str(row['Firstname']) + str(row['Surname']) + str(row['DOB'])
    # Generate a hash using SHA-256
    return hashlib.sha256(combined.encode()).hexdigest()

# Specify column widths based on the file structure
colspecs = [(7, 36), (36, 51)] # Column specs to extract Player and DOB

# Read the file
df = pd.read_fwf('https://afltables.com/afl/stats/biglists/bg10.txt', 
                 colspecs=colspecs, 
                 header=None, 
                 names=['Player', 'DOB'],
                 skiprows=2
                )

# Clean and process the dataframe
df['Player'] = df['Player'].str.strip()
df[['Firstname', 'Surname']] = df['Player'].str.split(n=1, expand=True)
df['DOB'] = df['DOB'].str.strip()
df['Player_Hashkey'] = df.apply(hash_row, axis=1)
df=df[['Player_Hashkey','Firstname','Surname','DOB']]
df['Load_Date']=datetime.now().date()
df['Record_Source']='https://afltables.com/afl/stats/biglists/bg10.txt'
df=df[['Player_Hashkey','Firstname','Surname','DOB','Load_Date','Record_Source']]


Load data ito H_Player

In [4]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='H_Player'
connection=sqlite3.connect(database=data_base)
df.to_sql(table_name, connection, index=False, if_exists='append')
connection.close()


Use the connection below to check H_Player

1.  There are duplicate first name,surname combinetions this will cause issue because it is not a complete business key

In [5]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(database=data_base)
query_ps= f''' 

with p_with_issue as (

select firstname,surname,count(1) from H_Player
where 1=1
group by firstname,surname
having count(1) >1)
select * from H_player hp
join  p_with_issue pwi
on   hp.firstname=pwi.firstname
and  hp.surname=pwi.surname
order by hp.Surname,hp.firstname,hp.dob
'''
df_out=pd.read_sql(query_ps,connection)
connection.close()
df_out

,Player_Hashkey,Firstname,Surname,DOB,Load_Date,Record_Source,firstname,surname,count(1)
0,54210a5f754054c7dbfb7a95e351fa66afcf3b4c35d4cd...,Gary,Ablett,1-Oct-1961,2024-10-13,https://afltables.com/afl/stats/biglists/bg10.txt,Gary,Ablett,2
1,2ec05a842a615714e61bc413b0f13e38397f1264777a77...,Gary,Ablett,14-May-1984,2024-10-13,https://afltables.com/afl/stats/biglists/bg10.txt,Gary,Ablett,2
2,359cab64a679f5ed8263a198b469f9333c98678dfc81d2...,Bill,Ahern,2-Mar-1865,2024-10-13,https://afltables.com/afl/stats/biglists/bg10.txt,Bill,Ahern,2
3,24623ebd3023d4cc10d508522167c27fd0781fda348758...,Bill,Ahern,25-Jun-1874,2024-10-13,https://afltables.com/afl/stats/biglists/bg10.txt,Bill,Ahern,2
4,0abd8bef79fa9e2d4366b6583ef7dabea5b5f07951a35d...,Frank,Aked,25-Nov-1902,2024-10-13,https://afltables.com/afl/stats/biglists/bg10.txt,Frank,Aked,2
...,...,...,...,...,...,...,...,...,...
994,b0c507950f0badd06bdd7f5965a3195b9d06e37cd90996...,Peter,Wood,27-Jan-1946,2024-10-13,https://afltables.com/afl/stats/biglists/bg10.txt,Peter,Wood,2
995,d56c9cfa473c5940297ec8bb861686c4ed4318c2a491a5...,Charlie,Young,10-Oct-1918,2024-10-13,https://afltables.com/afl/stats/biglists/bg10.txt,Charlie,Young,2
996,39577077ad832bb9aa2c7b321c305fab2234a8edbea208...,Charlie,Young,7-Dec-1877,2024-10-13,https://afltables.com/afl/stats/biglists/bg10.txt,Charlie,Young,2
997,57cc72ad86042e06323a55ef0d3fe23396f38d928b8d32...,George,Young,19-Oct-1878,2024-10-13,https://afltables.com/afl/stats/biglists/bg10.txt,George,Young,2
